# Regression with neural network in pyTorch
### Training data

In [1]:
import numpy as np
import torch as tr

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

### Convert the input target to tensor

In [2]:
input_data = tr.from_numpy(inputs)
target_data = tr.from_numpy(targets)
print(input_data)
print(target_data)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Weights and bias
``
Weights and bias initialized as random values. The first row of w and b are used to predict the first target variable. i.e., yield of apples and similarly the second for oranges
``

In [3]:
tr.random.manual_seed(42)
w = tr.randn(2, 3, requires_grad=True) # weights, need to grad is True
b = tr.randn(2, requires_grad=True) # bias, need to grad is True
print(w)
print(b)

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]], requires_grad=True)
tensor([ 2.2082, -0.6380], requires_grad=True)


### Define the model

In [4]:
def model(x):
    return x @ w.t() + b

In [5]:
### generate predictions

In [6]:
preds = model(input_data)
print(preds)

tensor([[  45.4987,  -67.0672],
        [  59.1878,  -90.4141],
        [  62.3595, -141.8688],
        [  50.7645,  -32.3210],
        [  54.2179, -105.5822]], grad_fn=<AddBackward0>)


In [7]:
# Compare the targets
print(target_data)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [8]:
# Loss function
### MSE loss

In [9]:
def mse(m, n):
    diff = m - n
    return tr.sum(diff*diff)/diff.numel()
# compute loss
loss = mse(preds, target_data)
print(loss)

tensor(19322.3477, grad_fn=<DivBackward0>)


In [10]:
### Compute gradiants

In [11]:
# compute the gradiants
loss.backward()

In [12]:
# gradiants for weights
print(w)
print(w.grad)

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]], requires_grad=True)
tensor([[ -1622.2405,  -2731.8169,  -1496.6278],
        [-14781.0176, -17480.2129, -10474.4824]])


In [13]:
w.grad

tensor([[ -1622.2405,  -2731.8169,  -1496.6278],
        [-14781.0176, -17480.2129, -10474.4824]])

In [14]:
with tr.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [15]:
# let's verify that the loss is actually lower
loss = mse(preds, target_data)
print(loss)

tensor(19322.3477, grad_fn=<DivBackward0>)


In [16]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [17]:
### Train the model using gradient descent

In [18]:
# Generate predictions
preds = model(input_data)
print(preds)

tensor([[ 49.1570, -40.0595],
        [ 64.0261, -54.8753],
        [ 68.2998, -99.5089],
        [ 54.1479,  -5.8505],
        [ 59.0077, -71.2684]], grad_fn=<AddBackward0>)


In [19]:
# Calculate the loss
loss = mse(preds, target_data)
print(loss)

tensor(13438.3047, grad_fn=<DivBackward0>)


In [20]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -1242.2932,  -2317.3828,  -1242.2361],
        [-11989.3066, -14471.1055,  -8619.6689]])
tensor([ -17.2723, -146.3125])


In [21]:
# Adjust weights & reset gradients
with tr.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [22]:
print(w)
print(b)

tensor([[ 0.3653,  0.1793,  0.2619],
        [ 0.4980, -0.8033,  0.0046]], requires_grad=True)
tensor([ 2.2086, -0.6347], requires_grad=True)


In [23]:
# Calculate loss
preds = model(input_data)
loss = mse(preds, target_data)
print(loss)

tensor(9467.9111, grad_fn=<DivBackward0>)


In [24]:
## Train for multiple epochs

In [25]:
# train for 100 epochs
for i in range(150):
    preds = model(input_data)
    loss = mse(preds, target_data)
    
    loss.backward()
    with tr.no_grad():
        w -= w.grad*1e-5
        b -= b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()

In [26]:
# Calculate loss
preds = model(input_data)
loss = mse(preds, target_data)
print(loss)

tensor(201.1342, grad_fn=<DivBackward0>)


In [27]:
preds

tensor([[ 60.9633,  75.7079],
        [ 82.6557, 102.2226],
        [111.5731, 120.7462],
        [ 40.7990,  68.7990],
        [ 91.6298, 103.1659]], grad_fn=<AddBackward0>)

In [28]:
target_data

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear Regression using PyTorch bulit-in

In [45]:
import torch.nn as nn

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = tr.from_numpy(inputs)
targets = tr.from_numpy(targets)

In [32]:
### Dataset and DataLoader

In [46]:
from torch.utils.data import TensorDataset

# define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [35]:
### DataLoader

In [47]:
from torch.utils.data import DataLoader

# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [48]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 74.,  66.,  43.],
        [103.,  43.,  36.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 88., 134.,  59.]])
tensor([[ 57.,  69.],
        [ 20.,  38.],
        [ 22.,  37.],
        [ 81., 101.],
        [118., 132.]])


In [49]:
### nn.Linear

In [52]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4428,  0.4737,  0.1663],
        [ 0.2391,  0.1826, -0.0100]], requires_grad=True)
Parameter containing:
tensor([ 0.4518, -0.4102], requires_grad=True)


In [53]:
# generate prediction
preds = model(inputs)
preds

tensor([[  7.0156,  28.8494],
        [ 12.4852,  36.7775],
        [ 35.0507,  44.2805],
        [-18.1943,  31.4626],
        [ 27.0153,  32.9168],
        [  6.0990,  28.9059],
        [ 12.1777,  36.5849],
        [ 34.7742,  44.5096],
        [-17.2777,  31.4061],
        [ 27.6245,  32.6676],
        [  6.7082,  28.6567],
        [ 11.5686,  36.8341],
        [ 35.3582,  44.4731],
        [-18.8034,  31.7118],
        [ 27.9319,  32.8603]], grad_fn=<AddmmBackward0>)

In [54]:
### Loss Function

In [56]:
# import nn.functional
import torch.nn.functional as F
# define loss function
loss_fn = F.mse_loss # mse_loss = bulit-in loss function

In [58]:
loss = loss_fn(preds, targets)
print(loss)

tensor(4250.6431, grad_fn=<MseLossBackward0>)


## Optimizer

In [61]:
# define optimizer
opt = tr.optim.SGD(model.parameters(), lr=1e-5)

### Train the model

In [74]:
# utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
      
    # repeat for given number of epoach
    for epoch in range(num_epochs):
        # repeat with batches of data
        for xb, yb in train_dl:
                
            # 1. Generate predictions
            pred = model(xb)
                
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
                
            # 3. compute gradiants
            loss.backward()
                
            # 4. Update parameters suing gradinants
            opt.step()
                
            # 5. repeat the gradiants to zero
            opt.zero_grad()
                
        # print the progress
        if (epoch+1) % 10 ==0:
            print('Epoch [{}/{}], loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))        

In [79]:
fit(150, model, loss_fn, opt, train_dl)

Epoch [10/150], loss: 0.8247
Epoch [20/150], loss: 2.3827
Epoch [30/150], loss: 1.9318
Epoch [40/150], loss: 1.3267
Epoch [50/150], loss: 1.8983
Epoch [60/150], loss: 1.8957
Epoch [70/150], loss: 1.9471
Epoch [80/150], loss: 1.4654
Epoch [90/150], loss: 1.8014
Epoch [100/150], loss: 1.4987
Epoch [110/150], loss: 0.9892
Epoch [120/150], loss: 1.2277
Epoch [130/150], loss: 1.2430
Epoch [140/150], loss: 0.9705
Epoch [150/150], loss: 1.3277


In [80]:
# compute the target
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [82]:
preds = model(inputs)
preds

tensor([[ 57.0511,  70.3155],
        [ 81.7502, 100.0017],
        [118.6618, 133.7326],
        [ 20.9337,  37.9667],
        [101.3927, 117.3920],
        [ 55.7960,  69.2299],
        [ 81.5611, 100.0096],
        [118.9263, 134.2903],
        [ 22.1888,  39.0524],
        [102.4587, 118.4856],
        [ 56.8620,  70.3235],
        [ 80.4951,  98.9160],
        [118.8508, 133.7246],
        [ 19.8677,  36.8731],
        [102.6478, 118.4776]], grad_fn=<AddmmBackward0>)

In [85]:
model(tr.tensor([[75, 63, 44.]]))

tensor([[53.4975, 67.3346]], grad_fn=<AddmmBackward0>)